### Import Libraries

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os


In [8]:
seed = 42

sns.set_style('darkgrid')
sns.set_palette('Set2')


### Load datset

#### Read the data

In [9]:
train = pd.read_csv("dataset/train.csv")

In [13]:
dataset = ["train", "test", "oil", "holidays_events", "stores", "transactions"]

In [14]:
for data in dataset:
    print(data)
    exec("{} = pd.read_csv('dataset/{}.csv')".format(data, data))

train
test
oil
holidays_events
stores
transactions


#### explore data